# Wan 2.1/2.2 - Generation Video Multilingue

**Module :** 02-Video-Advanced  
**Niveau :** Intermediaire  
**Technologies :** Wan 2.1/2.2 (Alibaba), diffusers, bitsandbytes  
**Duree estimee :** 45 minutes  
**VRAM :** ~10 GB  

## Objectifs d'Apprentissage

- [ ] Comprendre l'architecture Wan et ses variantes (2.1 / 2.2)
- [ ] Charger le modele Wan 2.1 T2V avec quantification optionnelle
- [ ] Generer des videos avec des prompts en francais et en anglais
- [ ] Explorer le controle de mouvement et les mouvements de camera
- [ ] Maitriser les options de resolution et de ratio d'aspect
- [ ] Comparer les capacites de Wan 2.1 vs 2.2
- [ ] Optimiser les prompts pour de meilleurs resultats

## Prerequis

- GPU avec 10+ GB VRAM (RTX 3060 12GB recommande)
- Notebooks 02-1 et 02-2 completes pour comparaison
- Packages : `diffusers>=0.32`, `transformers`, `torch`, `accelerate`, `imageio`, `imageio-ffmpeg`

**Navigation :** [<< 02-2](02-2-LTX-Video-Lightweight.ipynb) | [Index](../README.md) | [Suivant >>](02-4-SVD-Image-to-Video.ipynb)

In [ ]:
# Parametres Papermill - JAMAIS modifier ce commentaire

# Configuration notebook
notebook_mode = "interactive"        # "interactive" ou "batch"
skip_widgets = False               # True pour mode batch MCP
debug_level = "INFO"

# Parametres modele
model_id = "Wan-AI/Wan2.1-T2V-14B"  # Modele Wan 2.1 Text-to-Video
quantize = True                    # Quantification INT8 (recommande)
device = "cuda"                    # Device de calcul

# Parametres generation
num_frames = 16                    # Nombre de frames a generer
guidance_scale = 5.0               # CFG scale
num_inference_steps = 25           # Nombre d'etapes de debruitage
height = 480                       # Hauteur video
width = 832                        # Largeur video (ratio 16:9)
fps_output = 16                    # FPS de la video de sortie

# Configuration
run_generation = True              # Executer la generation
save_as_mp4 = True                 # Sauvegarder en MP4
save_results = True

In [ ]:
# Setup environnement et imports
import os
import sys
import json
import time
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import logging

warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Import helpers GenAI
GENAI_ROOT = Path.cwd()
while GENAI_ROOT.name != 'GenAI' and len(GENAI_ROOT.parts) > 1:
    GENAI_ROOT = GENAI_ROOT.parent

HELPERS_PATH = GENAI_ROOT / 'shared' / 'helpers'
if HELPERS_PATH.exists():
    sys.path.insert(0, str(HELPERS_PATH.parent))
    try:
        from helpers.genai_helpers import setup_genai_logging
        print("Helpers GenAI importes")
    except ImportError:
        print("Helpers GenAI non disponibles - mode autonome")

OUTPUT_DIR = GENAI_ROOT / 'outputs' / 'wan_video'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=getattr(logging, debug_level))
logger = logging.getLogger('wan_video')

print(f"Wan 2.1/2.2 - Generation Video Multilingue")
print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Frames : {num_frames}, Steps : {num_inference_steps}, CFG : {guidance_scale}")
print(f"Quantification : {'INT8' if quantize else 'FP16'}")

In [ ]:
# Chargement .env et verification GPU
from dotenv import load_dotenv

current_path = Path.cwd()
found_env = False
for _ in range(4):
    env_path = current_path / '.env'
    if env_path.exists():
        load_dotenv(env_path)
        print(f"Fichier .env charge depuis : {env_path}")
        found_env = True
        break
    current_path = current_path.parent

if not found_env:
    print("Aucun fichier .env trouve")

# Verification GPU
print("\n--- VERIFICATION GPU ---")
print("=" * 40)

import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_total = torch.cuda.get_device_properties(0).total_mem / 1024**3
    vram_free = (torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated(0)) / 1024**3
    
    print(f"GPU : {gpu_name}")
    print(f"VRAM totale : {vram_total:.1f} GB")
    print(f"VRAM libre : {vram_free:.1f} GB")
    print(f"CUDA : {torch.version.cuda}")
    
    if vram_total < 10:
        print(f"\nAttention : VRAM ({vram_total:.0f} GB) < 10 GB recommandes")
        quantize = True
        height = 320
        width = 576
        num_frames = 12
        print(f"  Resolution reduite a {width}x{height}, {num_frames} frames")
else:
    print("CUDA non disponible.")
    print("Wan necessite un GPU. Le notebook montrera le code sans executer.")
    run_generation = False
    device = "cpu"

# Verification des dependances
print("\n--- VERIFICATION DEPENDANCES ---")
print("=" * 40)

deps_ok = True

try:
    import diffusers
    print(f"diffusers : v{diffusers.__version__}")
except ImportError:
    print("diffusers NON INSTALLE (pip install diffusers>=0.32)")
    deps_ok = False

try:
    import transformers
    print(f"transformers : v{transformers.__version__}")
except ImportError:
    print("transformers NON INSTALLE")
    deps_ok = False

if quantize:
    try:
        import bitsandbytes as bnb
        print(f"bitsandbytes : v{bnb.__version__}")
    except ImportError:
        print("bitsandbytes NON INSTALLE (pip install bitsandbytes)")
        print("  Quantification INT8 non disponible")
        quantize = False

try:
    import imageio
    print(f"imageio : v{imageio.__version__}")
except ImportError:
    print("imageio NON INSTALLE")
    deps_ok = False

if not deps_ok:
    print("\nDependances manquantes. Le notebook montrera le code sans executer.")
    run_generation = False

print(f"\nDevice : {device}")
print(f"Generation activee : {run_generation}")
print(f"Quantification : {'INT8' if quantize else 'FP16'}")

## Section 1 : Architecture Wan 2.1/2.2

Wan est une famille de modeles de generation video developpes par Alibaba. La serie 2.1/2.2
se distingue par sa comprehension multilingue et son controle fin du mouvement.

| Composant | Description |
|-----------|-------------|
| **Architecture** | Transformer avec attention croisee spatio-temporelle |
| **Text encoder** | Multilingue (chinois, anglais, francais) |
| **Variantes** | T2V-1.3B (leger) / T2V-14B (haute qualite) |
| **Scheduler** | UniPC (rapide) ou DPM-Solver |

### Comparaison Wan 2.1 vs 2.2

| Aspect | Wan 2.1 | Wan 2.2 |
|--------|---------|----------|
| Qualite | Bonne | Amelioree |
| Coherence temporelle | Bonne | Tres bonne |
| Motion control | Basique | Camera paths avancees |
| Resolutions | 480p-720p | 480p-1080p |
| VRAM (14B, INT8) | ~10 GB | ~12 GB |

In [ ]:
# Chargement du pipeline Wan
pipe = None

if run_generation:
    print("\n--- CHARGEMENT DU PIPELINE ---")
    print("=" * 40)
    
    try:
        from diffusers import WanPipeline
        from diffusers.utils import export_to_video
        
        start_load = time.time()
        
        if quantize:
            from diffusers import BitsAndBytesConfig
            
            print(f"Chargement avec quantification INT8...")
            print(f"  Modele : {model_id}")
            
            quant_config = BitsAndBytesConfig(
                load_in_8bit=True
            )
            
            pipe = WanPipeline.from_pretrained(
                model_id,
                quantization_config=quant_config,
                torch_dtype=torch.float16
            )
        else:
            print(f"Chargement en FP16...")
            print(f"  Modele : {model_id}")
            
            pipe = WanPipeline.from_pretrained(
                model_id,
                torch_dtype=torch.float16
            )
        
        pipe = pipe.to(device)
        
        # Optimisations memoire
        pipe.enable_vae_slicing()
        pipe.enable_vae_tiling()
        try:
            pipe.enable_model_cpu_offload()
        except Exception:
            pass
        
        load_time = time.time() - start_load
        
        if device == "cuda":
            vram_used = torch.cuda.memory_allocated(0) / 1024**3
            print(f"  VRAM utilisee : {vram_used:.1f} GB")
        
        print(f"Pipeline charge en {load_time:.1f}s")
        print(f"  Quantification : {'INT8' if quantize else 'FP16'}")
        print(f"  Resolution : {width}x{height}")
        
    except Exception as e:
        print(f"Erreur chargement pipeline : {type(e).__name__}: {str(e)[:200]}")
        print("Le notebook continuera sans generation.")
        run_generation = False
        pipe = None
else:
    print("Chargement pipeline desactive")

## Section 2 : Generation avec prompts multilingues

L'un des avantages distinctifs de Wan est sa comprehension multilingue.
Nous allons tester des prompts en francais et en anglais pour comparer les resultats.

In [ ]:
# Generation avec prompts multilingues
print("\n--- PROMPTS MULTILINGUES ---")
print("=" * 40)

def generate_wan_video(prompt: str, negative_prompt: str = "",
                       seed: int = 42) -> Dict[str, Any]:
    """
    Genere une video avec Wan.
    
    Args:
        prompt: Description textuelle (FR, EN ou CN)
        negative_prompt: Elements a eviter
        seed: Graine aleatoire pour reproductibilite
    
    Returns:
        Dict avec frames, temps de generation et metadonnees
    """
    if pipe is None:
        return {"success": False, "error": "Pipeline non charge"}
    
    try:
        generator = torch.Generator(device=device).manual_seed(seed)
        
        if device == "cuda":
            torch.cuda.reset_peak_memory_stats()
        
        start_time = time.time()
        
        output = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt or "low quality, blurry, distorted, watermark",
            num_frames=num_frames,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            height=height,
            width=width,
            generator=generator
        )
        
        gen_time = time.time() - start_time
        frames = output.frames[0]
        
        result = {
            "success": True,
            "frames": frames,
            "generation_time": gen_time,
            "time_per_frame": gen_time / num_frames,
            "prompt": prompt,
            "seed": seed,
            "params": {
                "num_frames": num_frames,
                "guidance_scale": guidance_scale,
                "num_inference_steps": num_inference_steps,
                "height": height,
                "width": width
            }
        }
        
        if device == "cuda":
            result["vram_peak"] = torch.cuda.max_memory_allocated(0) / 1024**3
        
        return result
        
    except Exception as e:
        return {"success": False, "error": f"{type(e).__name__}: {str(e)[:200]}"}


# Test bilingue FR/EN
bilingual_prompts = [
    {
        "text": "Un chat roux dort paisiblement sur un rebord de fenetre ensoleille, lumiere douce, atmosphere calme",
        "lang": "FR",
        "label": "Chat (FR)"
    },
    {
        "text": "A ginger cat sleeping peacefully on a sunny windowsill, soft light, calm atmosphere",
        "lang": "EN",
        "label": "Cat (EN)"
    }
]

bilingual_results = []

if run_generation:
    for p_idx, prompt_info in enumerate(bilingual_prompts):
        print(f"\nGeneration {p_idx + 1}/{len(bilingual_prompts)} : {prompt_info['label']}")
        print(f"  Prompt ({prompt_info['lang']}) : {prompt_info['text'][:70]}...")
        
        result = generate_wan_video(prompt_info['text'], seed=42)
        
        if result['success']:
            print(f"  Temps : {result['generation_time']:.1f}s")
            bilingual_results.append({
                "label": prompt_info['label'],
                "lang": prompt_info['lang'],
                "frames": result['frames'],
                "time": result['generation_time']
            })
            
            if save_as_mp4:
                mp4_path = OUTPUT_DIR / f"wan_{prompt_info['lang'].lower()}_demo.mp4"
                export_to_video(result['frames'], str(mp4_path), fps=fps_output)
        else:
            print(f"  Erreur : {result['error']}")
    
    # Affichage comparatif FR vs EN
    if len(bilingual_results) == 2:
        fig, axes = plt.subplots(2, 4, figsize=(16, 8))
        
        for v_idx, br in enumerate(bilingual_results):
            frame_indices = np.linspace(0, len(br['frames']) - 1, 4, dtype=int)
            for f_idx, fi in enumerate(frame_indices):
                axes[v_idx][f_idx].imshow(br['frames'][fi])
                axes[v_idx][f_idx].axis('off')
                if f_idx == 0:
                    axes[v_idx][f_idx].set_ylabel(br['label'], fontsize=11, fontweight='bold')
        
        plt.suptitle("Comparaison FR vs EN - Meme sujet", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f"\n{'Langue':<15} {'Temps (s)':<12}")
        print("-" * 27)
        for br in bilingual_results:
            print(f"  {br['label']:<15} {br['time']:<12.1f}")
else:
    print("Generation desactivee")
    print("\nWan supporte les prompts en :")
    print("  - Francais : 'Un coucher de soleil sur la mer, couleurs chaudes'")
    print("  - Anglais : 'A sunset over the sea, warm colors'")
    print("  - Chinois : nativement supporte")

### Interpretation : Prompts multilingues

| Aspect | Francais | Anglais |
|--------|---------|----------|
| Qualite | Bonne | Bonne (legerement meilleure) |
| Comprehension | Concepts principaux captes | Plus de nuances |
| Temps | Similaire | Similaire |

**Points cles** :
1. Wan comprend bien le francais grace a son encodeur multilingue
2. Les prompts en anglais donnent generalement des resultats legerement meilleurs (donnees d'entrainement)
3. Il est possible de melanger les langues dans un meme prompt

## Section 3 : Controle de mouvement

Wan permet de controler le type de mouvement genere via les prompts.
Certains termes cles orientent le modele vers des mouvements de camera specifiques.

In [ ]:
# Controle de mouvement et camera
if run_generation and pipe is not None:
    print("\n--- CONTROLE DE MOUVEMENT ---")
    print("=" * 40)
    
    motion_prompts = [
        {
            "text": "a slow pan across a beautiful Japanese garden with cherry blossoms, smooth camera movement, serene",
            "label": "Pan lateral",
            "motion": "Panoramique"
        },
        {
            "text": "camera slowly zooming into a detailed oil painting of a medieval castle, revealing intricate details",
            "label": "Zoom avant",
            "motion": "Zoom"
        },
        {
            "text": "aerial drone shot flying over a coastal city at golden hour, birds eye view, cinematic",
            "label": "Vol aerien",
            "motion": "Drone"
        }
    ]
    
    motion_results = []
    
    for p_idx, prompt_info in enumerate(motion_prompts):
        print(f"\nGeneration {p_idx + 1}/{len(motion_prompts)} : {prompt_info['label']}")
        print(f"  Type de mouvement : {prompt_info['motion']}")
        
        result = generate_wan_video(prompt_info['text'], seed=42 + p_idx)
        
        if result['success']:
            print(f"  Temps : {result['generation_time']:.1f}s")
            motion_results.append({
                "label": prompt_info['label'],
                "motion": prompt_info['motion'],
                "frames": result['frames'],
                "time": result['generation_time']
            })
            
            if save_as_mp4:
                mp4_path = OUTPUT_DIR / f"wan_motion_{prompt_info['motion'].lower()}.mp4"
                export_to_video(result['frames'], str(mp4_path), fps=fps_output)
        else:
            print(f"  Erreur : {result['error']}")
    
    # Affichage comparatif des mouvements
    if motion_results:
        n_motions = len(motion_results)
        n_preview = 4
        fig, axes = plt.subplots(n_motions, n_preview, figsize=(3.5 * n_preview, 3 * n_motions))
        if n_motions == 1:
            axes = [axes]
        
        for v_idx, mr in enumerate(motion_results):
            frame_indices = np.linspace(0, len(mr['frames']) - 1, n_preview, dtype=int)
            for f_idx, fi in enumerate(frame_indices):
                axes[v_idx][f_idx].imshow(mr['frames'][fi])
                axes[v_idx][f_idx].axis('off')
                if f_idx == 0:
                    axes[v_idx][f_idx].set_ylabel(mr['label'], fontsize=10, fontweight='bold')
        
        plt.suptitle("Controle de mouvement - Wan", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f"\n{'Mouvement':<15} {'Type':<15} {'Temps (s)':<12}")
        print("-" * 42)
        for mr in motion_results:
            print(f"  {mr['label']:<15} {mr['motion']:<15} {mr['time']:<12.1f}")
else:
    print("Controle de mouvement : generation desactivee")
    print("\nMots-cles de mouvement pour Wan :")
    print("  - 'slow pan' : panoramique lateral")
    print("  - 'zoom in/out' : rapprochement/eloignement")
    print("  - 'aerial drone shot' : vue aerienne")
    print("  - 'tracking shot' : suivi de sujet")
    print("  - 'dolly shot' : deplacement lineaire de camera")

### Interpretation : Controle de mouvement

| Mot-cle prompt | Mouvement obtenu | Efficacite |
|---------------|-----------------|------------|
| "slow pan" | Panoramique lateral fluide | Elevee |
| "zoom into" | Rapprochement progressif | Bonne |
| "aerial drone" | Survol plongeant | Bonne |
| "tracking shot" | Suivi lateral d'un sujet | Moyenne |
| "dolly" | Deplacement lineaire | Moyenne |

**Points cles** :
1. Les termes cinematographiques en anglais donnent le meilleur controle
2. Les mouvements simples (pan, zoom) sont mieux geres que les complexes
3. Combiner un mouvement de camera avec une description de scene donne les meilleurs resultats

## Section 4 : Resolution et ratio d'aspect

Wan supporte differents ratios d'aspect. Le choix du ratio influence la composition de la scene.

In [ ]:
# Test de differents ratios d'aspect
if run_generation and pipe is not None:
    print("\n--- RATIOS D'ASPECT ---")
    print("=" * 40)
    
    aspect_prompt = "a majestic lighthouse standing on a cliff, waves crashing, dramatic sky, golden hour"
    
    aspect_configs = [
        {"w": 480, "h": 480, "label": "1:1 (480x480)"},
        {"w": 832, "h": 480, "label": "16:9 (832x480)"},
        {"w": 480, "h": 832, "label": "9:16 (480x832)"},
    ]
    
    aspect_results = []
    
    for cfg in aspect_configs:
        print(f"\nTest : {cfg['label']}")
        
        try:
            if device == "cuda":
                torch.cuda.reset_peak_memory_stats()
            
            generator = torch.Generator(device=device).manual_seed(42)
            start_time = time.time()
            
            output = pipe(
                prompt=aspect_prompt,
                negative_prompt="low quality, blurry, distorted",
                num_frames=12,
                guidance_scale=5.0,
                num_inference_steps=20,
                height=cfg['h'],
                width=cfg['w'],
                generator=generator
            )
            
            gen_time = time.time() - start_time
            frames = output.frames[0]
            
            vram_peak = 0
            if device == "cuda":
                vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3
            
            aspect_results.append({
                "label": cfg['label'],
                "frames": frames,
                "time": gen_time,
                "vram_peak": vram_peak
            })
            
            print(f"  Temps : {gen_time:.1f}s, VRAM pic : {vram_peak:.1f} GB")
            
        except Exception as e:
            print(f"  Erreur : {type(e).__name__}: {str(e)[:100]}")
    
    # Affichage : premiere et derniere frame de chaque ratio
    if aspect_results:
        fig, axes = plt.subplots(len(aspect_results), 2, figsize=(10, 4 * len(aspect_results)))
        if len(aspect_results) == 1:
            axes = [axes]
        
        for v_idx, ar in enumerate(aspect_results):
            axes[v_idx][0].imshow(ar['frames'][0])
            axes[v_idx][0].set_title(f"{ar['label']} - Frame 1", fontsize=10)
            axes[v_idx][0].axis('off')
            
            axes[v_idx][1].imshow(ar['frames'][-1])
            axes[v_idx][1].set_title(f"{ar['label']} - Derniere frame", fontsize=10)
            axes[v_idx][1].axis('off')
        
        plt.suptitle("Ratios d'aspect - Wan", fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print(f"\n{'Ratio':<20} {'Temps (s)':<12} {'VRAM (GB)':<12}")
        print("-" * 44)
        for ar in aspect_results:
            print(f"  {ar['label']:<20} {ar['time']:<12.1f} {ar['vram_peak']:<12.1f}")
else:
    print("Test ratio d'aspect : generation desactivee")
    print("\nRatios supportes par Wan :")
    print("  1:1  (480x480) : Reseaux sociaux carre")
    print("  16:9 (832x480) : YouTube, cinema")
    print("  9:16 (480x832) : TikTok, Reels, Stories")

### Interpretation : Ratios d'aspect

| Ratio | Usage | Particularite |
|-------|-------|---------------|
| 1:1 (carre) | Instagram, prototypage | Composition centree |
| 16:9 (paysage) | YouTube, cinema | Composition large, horizon |
| 9:16 (portrait) | TikTok, Stories | Composition verticale, sujet central |

**Points cles** :
1. Le ratio d'aspect influence la composition automatique de la scene
2. Les paysages fonctionnent mieux en 16:9, les portraits en 9:16
3. La VRAM est proportionnelle au nombre total de pixels (W x H)

## Bonnes pratiques et prompt engineering

### Structure d'un bon prompt Wan

| Element | Exemple | Importance |
|---------|---------|------------|
| Sujet | "a majestic eagle" | Essentiel |
| Action | "soaring through clouds" | Essentiel |
| Camera | "aerial tracking shot" | Recommande |
| Eclairage | "golden hour, dramatic light" | Recommande |
| Style | "cinematic, 4K, high quality" | Optionnel |

### Prompts en francais vs anglais

| Approche | Avantage | Inconvenient |
|----------|----------|-------------|
| Tout FR | Naturel pour le redacteur | Moins de nuances captees |
| Tout EN | Meilleure comprehension | Moins intuitif |
| Mixte | Compromis optimal | Necessite expertise bilingue |

In [ ]:
# Mode interactif
if notebook_mode == "interactive" and not skip_widgets:
    print("\n--- MODE INTERACTIF ---")
    print("=" * 40)
    print("Entrez votre propre prompt (francais ou anglais) pour generer une video Wan.")
    print("(Laissez vide pour passer a la suite)")
    
    try:
        user_prompt = input("\nVotre prompt (FR ou EN) : ").strip()
        
        if user_prompt and run_generation and pipe is not None:
            print(f"\nGeneration en cours...")
            result_user = generate_wan_video(user_prompt, seed=123)
            
            if result_user['success']:
                print(f"Generation reussie en {result_user['generation_time']:.1f}s")
                
                n_display = min(8, len(result_user['frames']))
                fig, axes = plt.subplots(1, n_display, figsize=(2.5 * n_display, 3))
                if n_display == 1:
                    axes = [axes]
                indices = np.linspace(0, len(result_user['frames']) - 1, n_display, dtype=int)
                for ax, idx in zip(axes, indices):
                    ax.imshow(result_user['frames'][idx])
                    ax.set_title(f"Frame {idx+1}", fontsize=8)
                    ax.axis('off')
                plt.suptitle(f"Votre video : {user_prompt[:50]}...", fontweight='bold')
                plt.tight_layout()
                plt.show()
                
                if save_as_mp4:
                    user_mp4 = OUTPUT_DIR / "user_generation.mp4"
                    export_to_video(result_user['frames'], str(user_mp4), fps=fps_output)
                    print(f"MP4 sauvegarde : {user_mp4.name}")
            else:
                print(f"Erreur : {result_user['error']}")
        elif user_prompt:
            print("Generation non disponible (pipeline non charge)")
        else:
            print("Mode interactif ignore")
    
    except (KeyboardInterrupt, EOFError) as e:
        print(f"\nMode interactif interrompu ({type(e).__name__})")
    except Exception as e:
        error_type = type(e).__name__
        if "StdinNotImplemented" in error_type or "input" in str(e).lower():
            print("\nMode interactif non disponible (execution automatisee)")
        else:
            print(f"\nErreur inattendue : {error_type} - {str(e)[:100]}")
            print("Passage a la suite du notebook")
else:
    print("\nMode batch - Interface interactive desactivee")

In [ ]:
# Statistiques de session et prochaines etapes
print("\n--- STATISTIQUES DE SESSION ---")
print("=" * 40)

print(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Mode : {notebook_mode}")
print(f"Modele : {model_id}")
print(f"Quantification : {'INT8' if quantize else 'FP16'}")
print(f"Device : {device}")
print(f"Parametres : {num_frames} frames, {num_inference_steps} steps, CFG={guidance_scale}")
print(f"Resolution : {width}x{height}")

if device == "cuda" and torch.cuda.is_available():
    vram_peak = torch.cuda.max_memory_allocated(0) / 1024**3
    print(f"VRAM pic session : {vram_peak:.1f} GB")

if save_results and OUTPUT_DIR.exists():
    generated_files = list(OUTPUT_DIR.glob('*'))
    print(f"\nFichiers generes ({len(generated_files)}) :")
    for f in sorted(generated_files):
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name} ({size_kb:.1f} KB)")

# Liberation VRAM
if pipe is not None:
    del pipe
    if device == "cuda":
        torch.cuda.empty_cache()
        print(f"\nVRAM liberee")

print(f"\n--- PROCHAINES ETAPES ---")
print(f"1. Notebook 02-4 : SVD - Stable Video Diffusion (animation d'images statiques)")
print(f"2. Module 03-1 : Comparaison benchmark de tous les modeles (AnimateDiff, HunyuanVideo, LTX, Wan, SVD)")
print(f"3. Module 03-2 : Orchestration de pipelines text -> image -> video")
print(f"4. Tester Wan 2.2 quand disponible pour les ameliorations de qualite")

print(f"\nNotebook 02-3 Wan Video Generation termine - {datetime.now().strftime('%H:%M:%S')}")